# 네이버 블로그 웹 페이지 구조

네이버 블로그는 자체적으로 백업기능을 제공하긴 하는데 PDF 파일로만 백업이 되고 그 백업 기능역시 실제 포스트와는 많이 다른 형태로 저장 됩니다. 그래서 파이썬을 활용하여 네이버 블로그 백업 프로그램을 만들어 보도록 하겠습니다. 먼저 네이버 블로그 백업을 하기 위해선 블로그 웹페이지가 어떤 구조로 되어있는지를 알고 그것을 어떤 구조로 백업할지를 계획해야 합니다.

<font color="red">웹페이지는 항상 수시로 업데이트 되며 구조가 자주 바뀔 수 있습니다. 그러면 방금전까지 잘 동작하던 프로그램도 어느순간 오류가 발생할 수 밖에 없습니다. 그렇기 때문에 크롤링 특성상 기능을 이해해야 차후에 수정된 사이트나 웹페이지에 맞게 대응하여 코드를 수정할 수 있습니다. 강좌중 이해가 가지 않는 부분이 있으면 꼭! 인프런 질문답변 게시판을 활용하시길 바랍니다.</font>

## URL

일단 네이버 블로그는 iframe 내에서 보여지기 때문에 블로그의 주소가 노출되지 않습니다. 일단 블로그는 카테고리가 있고 그 카테고리에 해당하는 포스트 게시물이 존재합니다. 개발자도구를 활용하여 블로그의 블로그 메뉴를 클릭해보고 카테고리가 어떻게 출력되는지를 확인해봐야 합니다.

![2.jpg](images/2.jpg)

크롬 개발자도구를 열어 블로그 페이지를 로드 하면 WidgetListAsync.nhn 이라는 페이지가 xhr 형태로 읽혀지는것을 확인할 수 있습니다. 이렇게 내부적으로 Ajax 통신을 하여 카테고리 정보를 불러옵니다. 우리는 최초 이 카테고리 정보를 파싱하는 기능을 작성할 예정입니다.

![img.jpg](images/12.jpg)

WidgetListAsync.nhn 주소의 Headers 탭을 눌러 하단의 Query String Parameters 정보를 살펴보면 WidgetListAsync.nhn 주소에 여러가지 값을 쿼리하는것을 알 수 있습니다. 네이버 블로그를 개설해서 블로그 디자인 설정을 해보면 네이버의 카테고리, 방문객 표시 등의 요소들을 "위젯" 이라고 표현하는것을 알 수 있습니다. 위의 이미지를 보면 enableWidgetKeys 라는 이름의 변수에 이런 여러 종류의 위젯 정보를 요청한다고 추측해볼 수 있습니다.

In [ ]:
import requests
import hjson
from bs4 import BeautifulSoup

# 남박사의 블로그 아이디
naver_id = "nkj2001"
url = "https://blog.naver.com/WidgetListAsync.nhn?blogId={}&enableWidgetKeys=category".format(naver_id)

header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
    "Referer": "https://blog.naver.com/PostList.nhn?blogId={}&categoryNo=0&from=postList".format(naver_id)
}

category = {}
_r = requests.get(url, headers=header)
_json = hjson.loads(_r.text)
_category_str = _json.get("category").get("content")
bs = BeautifulSoup(_category_str, "lxml")
links = bs.select("a")
for l in links:
    _href = l.get("href")
    if _href is None or _href == "#":
        continue
    _text = l.text
    _id = l.get("id")
    _id = _id.replace("category", "")
    if category.get(_id) is None:
        category[_id] = _text

print(category)

### 주소 설정

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
# 남박사의 블로그 아이디
naver_id = "nkj2001"
url = "https://blog.naver.com/WidgetListAsync.nhn?blogId={}&enableWidgetKeys=category".format(naver_id)
</pre>
WidgetListAsync.nhn 주소에 우리는 블로그 아이디와 카테고리 정보만 요청하기 위해서 ```blogId=블로그아이디&enableWidgetKeys=category``` 처럼 ```url``` 변수값을 설정했습니다.


### 헤더값 설정

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
    "Referer": "https://blog.naver.com/PostList.nhn?blogId={}&categoryNo=0&from=postList".format(naver_id)
}
</pre>

카테고리 정보를 WidgetListAsync.nhn 주소를 통해 정상적으로 가져오기 위해선 반드시 header 값에 User-Agent 값과 Referer 값을 넘겨주어야 합니다. 그렇지 않으면 아무런 데이터를 얻어낼 수 없습니다.

### hjson 라이브러리

그리고 실제 WidgetListAsync.nhn 에서 리턴해주는 데이터는 json 이지만 파이썬 requests.get().json() 이나 json 라이브러리로는 읽어볼 수가 없습니다. 파이썬에서는 Json 형태가 "키": "값" 형태야 하는데 WidgetListAsync.nhn 에서 리턴해주는 값의 키 에는 쌍따옴표가 없습니다. 그래서 우리는 hjson 이라는 라이브러리를 사용해서 이 문제를 해결하도록 하겠습니다. 

> pip install hjson

### 카테고리

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
category = {}
_r = requests.get(url, headers=header)
_json = hjson.loads(_r.text)
_category_str = _json.get("category").get("content")
</pre>

카테고리 정보는 위에서 얻은 json 데이터의 category 키의 값으로 존재하는데 이 값이 html 태그 뭉치 입니다. 

![img](images/13.jpg)

위의 코드에서 ```_category_str``` 을 출력해보면 위의 이미지에서 처럼 HTML 문자열 형태로 되어있습니다. 

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
bs = BeautifulSoup(_category_str, "lxml")
links = bs.select("a")
for l in links:
    _href = l.get("href")
    if _href is None or _href == "#":
        continue
    _text = l.text
    _id = l.get("id")
    _id = _id.replace("category", "")
    if category.get(_id) is None:
        category[_id] = _text

print(category)
</pre>

이 HTML 텍스트 전체를 파싱하기 위해서 BeautifulSoup 라이브러리를 사용해 a 링크 태그만 추출합니다. 이렇게 추출된 카테고리 값을 dict 자료형으로 키는 카테고리 넘버, 값은 카테고리 이름으로 저장해놓습니다. 차후에 category[카테고리넘버] 를 통해 카테고리명을 구해오기 위해 이렇게 저장해놓습니다.